In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3

In [25]:
%load_ext sql

In [93]:
%sql sqlite:///starter_code.sqlite

u'Connected: None@starter_code.sqlite'

In [22]:
!ls ~/DSI-CHI-1/lessons/week-06/2.2-sql-joins-lab/assets/datasets/yelp/

businesses.csv  inspections.csv legend.csv      violations.csv


In [21]:
%pwd

u'/Users/christophereppig/DSI_week3'

In [27]:
%cd ~/DSI-CHI-1/lessons/week-06/2.2-sql-joins-lab/assets/datasets/yelp/

/Users/christophereppig/DSI-CHI-1/lessons/week-06/2.2-sql-joins-lab/assets/datasets/yelp


In [26]:
sqlite_db = 'starter_code.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [28]:
%ls

businesses.csv*  inspections.csv* legend.csv*      violations.csv*


In [30]:
businesses = pd.read_csv('businesses.csv')

In [31]:
inspections = pd.read_csv('inspections.csv')
legend = pd.read_csv('legend.csv')
violations = pd.read_csv('violations.csv')

In [33]:
businesses.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044


In [34]:
inspections.head(1)

,business_id,score,date,type
0,10,94,20140729,routine


In [35]:
legend.head(1)

,Minimum_Score,Maximum_Score,Description
0,0,70,Poor


In [36]:
violations.head(1)

,business_id,date,description
0,10,20140729,Insufficient hot water or running water [ dat...


In [37]:
businesses.columns

Index([u'business_id', u'name', u'address', u'city', u'state', u'postal_code',
       u'latitude', u'longitude', u'phone_number'],
      dtype='object')

In [38]:
create_business = "CREATE TABLE businesses (business_id varchar PRIMARY KEY, name varchar, address varchar, city varchar, state varchar, postal_code varchar, latitude varchar, longitude varchar, phone_number varchar);"

c.execute(create_business)

In [39]:
inspections.columns

Index([u'business_id', u'score', u'date', u'type'], dtype='object')

In [41]:
create_inspections = "CREATE TABLE inspections(business_id varchar PRIMARY KEY, score varchar, date varchar, type varchar);"

In [42]:
c.execute(create_inspections)

In [43]:
legend.columns

Index([u'Minimum_Score', u'Maximum_Score', u'Description'], dtype='object')

In [44]:
## column name, data type
c.execute("CREATE TABLE legend (minimum_score varchar, maximum_score varchar, description varchar);")

In [45]:
violations.columns

Index([u'business_id', u'date', u'description'], dtype='object')

In [46]:
c.execute("CREATE TABLE violations (business_id varchar PRIMARY KEY, date varchar, description varchar);")

In [48]:
## submits any SQL commands I've previously made
conn.commit()

In [50]:
results = c.execute("SELECT * FROM sqlite_master WHERE type='table';")

In [51]:
results.fetchall()

[(u'table',
  u'businesses',
  u'businesses',
  2,
  u'CREATE TABLE businesses (business_id varchar PRIMARY KEY, name varchar, address varchar, city varchar, state varchar, postal_code varchar, latitude varchar, longitude varchar, phone_number varchar)'),
 (u'table',
  u'inspections',
  u'inspections',
  4,
  u'CREATE TABLE inspections(business_id varchar PRIMARY KEY, score varchar, date varchar, type varchar)'),
 (u'table',
  u'legend',
  u'legend',
  6,
  u'CREATE TABLE legend (minimum_score varchar, maximum_score varchar, description varchar)'),
 (u'table',
  u'violations',
  u'violations',
  7,
  u'CREATE TABLE violations (business_id varchar PRIMARY KEY, date varchar, description varchar)')]

In [52]:
results = c.execute("SELECT * FROM inspections;")
results.fetchall()

[]

## ^^ there is no data in there yet and that's okay

In [53]:
legend.as_matrix()

array([[0, 70, 'Poor'],
       [71, 85, 'Needs Improvement'],
       [86, 90, 'Adequate'],
       [91, 100, 'Good']], dtype=object)

In [54]:
list(legend.to_records())   ## gives list of tuples 

[(0, 0, 70, 'Poor'),
 (1, 71, 85, 'Needs Improvement'),
 (2, 86, 90, 'Adequate'),
 (3, 91, 100, 'Good')]

In [55]:
legend.values

array([[0, 70, 'Poor'],
       [71, 85, 'Needs Improvement'],
       [86, 90, 'Adequate'],
       [91, 100, 'Good']], dtype=object)

In [56]:
c.executemany("INSERT INTO legend VALUES (?, ?, ?)", legend.values)  ## each ? creates a column, after the comma is what to put in there

In [58]:
## shows that it worked

results = c.execute("SELECT * FROM legend;")
results.fetchall()

[(u'0', u'70', u'Poor'),
 (u'71', u'85', u'Needs Improvement'),
 (u'86', u'90', u'Adequate'),
 (u'91', u'100', u'Good')]

In [62]:
violations.columns

Index([u'business_id', u'date', u'description'], dtype='object')

In [66]:
## the primary key wan't all unique values. we had to erase the table and start again
c.execute("DROP TABLE IF EXISTS violations;")     ## IF EXISTS only drops the table if it exists
c.execute("CREATE TABLE violations (business_id varchar, date varchar, description varchar);")
c.executemany("INSERT INTO violations VALUES (?, ?, ?)", violations.values)
conn.commit()

In [68]:
results = c.execute("SELECT * FROM violations;")
results.fetchall()

[(u'10',
  u'20140729',
  u'Insufficient hot water or running water  [ date violation corrected: 8/7/2014 ]'),
 (u'10',
  u'20140729',
  u'Unapproved or unmaintained equipment or utensils  [ date violation corrected: 8/7/2014 ]'),
 (u'10',
  u'20140114',
  u'Inadequate and inaccessible handwashing facilities  [ date violation corrected: 1/24/2014 ]'),
 (u'10',
  u'20140114',
  u'Unclean or degraded floors walls or ceilings  [ date violation corrected: 1/24/2014 ]'),
 (u'10',
  u'20140114',
  u'Improper storage of equipment utensils or linens  [ date violation corrected: 1/24/2014 ]'),
 (u'19', u'20141110', u'Improper storage of equipment utensils or linens'),
 (u'19',
  u'20141110',
  u'Inadequate food safety knowledge or lack of certified food safety manager'),
 (u'19',
  u'20140214',
  u'Inadequately cleaned or sanitized food contact surfaces  [ date violation corrected: 2/14/2014 ]'),
 (u'19',
  u'20140214',
  u'Permit license or inspection report not posted  [ date violation correc

In [69]:
results = c.execute("SELECT COUNT(*) FROM violations;")
results.fetchall()

[(39917,)]

In [71]:
inspections.columns

Index([u'business_id', u'score', u'date', u'type'], dtype='object')

In [73]:
c.execute("DROP TABLE IF EXISTS inspections;")
create_inspections = "CREATE TABLE inspections(business_id varchar, score varchar, date varchar, type varchar);"
c.execute(create_inspections)
c.executemany("INSERT INTO inspections VALUES (?, ?, ?, ?);", inspections.values)
conn.commit()

In [74]:
c.execute("SELECT COUNT(*) FROM inspections;").fetchall()

[(15431,)]

In [83]:
print businesses.columns
print len(businesses.columns)

Index([u'business_id', u'name', u'address', u'city', u'state', u'postal_code',
       u'latitude', u'longitude', u'phone_number'],
      dtype='object')
9


In [90]:
c.execute("DROP TABLE IF EXISTS businesses;")
create_business = "CREATE TABLE businesses (business_id varchar, name varchar, address varchar, city varchar, state varchar, postal_code varchar, latitude varchar, longitude varchar, phone_number varchar);"

c.execute(create_business)
c.executemany("INSERT INTO businesses VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);", 
              businesses.values)
conn.commit()

ProgrammingError: You must not use 8-bit bytestrings unless you use a text_factory that can interpret 8-bit bytestrings (like text_factory = str). It is highly recommended that you instead just switch your application to Unicode strings.

In [89]:
c.execute("SELECT COUNT(*) FROM businesses;").fetchall()

[(3177,)]

In [94]:
%%sql

SELECT * FROM businesses LIMIT 10;

(sqlite3.OperationalError) no such table: businesses [SQL: u'SELECT * FROM businesses LIMIT 10;']
